# Enterprise Image Analysis with AWS Rekognition

## Key Benefits
- Scalable for large data volumes
- Cost-effective pay-as-you-go model
- High accuracy with advanced ML models
- Easy API integration

## Enterprise Applications
1. Security and surveillance
2. Content moderation
3. Customer engagement
4. Media asset management
5. Fraud prevention

## AWS Service Integration
- Amazon S3: Image/video storage
- AWS Lambda: Serverless compute
- Amazon DynamoDB: Result storage
- Amazon CloudWatch: Monitoring
- Amazon SNS: Notifications

## Impact
Enables enterprises to build robust, scalable solutions for image/video analysis, enhancing operations from security to customer experience.

In [1]:
import boto3
import csv
import io
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import time


# Step 1: Load AWS Access Keys from the CSV file
with open('/Users/pawanbtw/Downloads/img-rekognition_accessKeys.csv', 'r') as file:
    next(file)
    reader = csv.reader(file)
    for line in reader:
        access_key_id = line[0]
        secret_access_key = line[1]

# Step 2: Initialize the Rekognition client
client = boto3.client('rekognition', region_name='us-east-1',
                      aws_access_key_id=access_key_id,
                      aws_secret_access_key=secret_access_key)

# Step 3: Image file to process
#photo = '/Users/pawanbtw/Desktop/run.jpg'
photo = '/Users/pawanbtw/Desktop/stevejobs2.jpg'
#photo = '/Users/pawanbtw/Desktop/monalisa.jpg'

with open(photo, 'rb') as image_file:
    source_bytes = image_file.read()

# Step 4: Call AWS Rekognition to detect labels
detect_objects = client.detect_labels(Image={'Bytes': source_bytes})

# Step 5: Load the specified font file
font_path = Path('/Users/pawanbtw/Downloads/Roboto_Condensed/RobotoCondensed-Italic-VariableFont_wght.ttf')
font = ImageFont.truetype(str(font_path), 30)

# Step 6: Open the image with Pillow
image = Image.open(io.BytesIO(source_bytes))
draw = ImageDraw.Draw(image)

# Step 7: Draw bounding boxes and labels
for label in detect_objects['Labels']:
    print(label["Name"])
    print("Confidence: ", label["Confidence"])
    
    for instance in label.get('Instances', []):
        if 'BoundingBox' in instance:
            box = instance["BoundingBox"]
            
            left = image.width * box['Left']
            top = image.height * box['Top']
            width = image.width * box['Width']
            height = image.height * box['Height']
            
            # Define points for the bounding box
            points = (
                (left, top),
                (left + width, top),
                (left + width, top + height),
                (left, top + height),
                (left, top)
            )
            draw.line(points, width=5, fill="#16ded7")

            # Draw the label above the bounding box
            shape = [(left - 2, top - 35), (left + width + 2, top)]
            draw.rectangle(shape, fill="#16ded7")
            draw.text((left + 10, top - 30), label["Name"], fill="#000000", font=font)

# Step 8: Display the image with bounding boxes
image.show()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/pawanbtw/Desktop/stevejobs2.jpg'

### This markdown provides a concise explanation of the code's purpose and includes a Python code block for the S3 upload functionality. It highlights the use of boto3 for AWS interactions and mentions the organization of processed images in a dedicated folder within the S3 bucket.

In [6]:
# Step 9: Save the processed image to an S3 bucket
s3_client = boto3.client('s3', region_name='us-east-1', 
                         aws_access_key_id=access_key_id,
                         aws_secret_access_key=secret_access_key)

# Convert the processed image back to bytes
image_byte_array = io.BytesIO()
image.save(image_byte_array, format="JPEG")
image_byte_array.seek(0)  # Rewind the byte stream to the beginning

# Define the S3 bucket and the folder path within the bucket

bucket_name = 'video2-analysis-bucket'  
folder_name = 'processed-images'     
file_name = 'processed_image.jpg'   

object_key = f"{folder_name}/{file_name}"


s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=image_byte_array, ContentType='image/jpeg')

print(f"Image saved to S3 bucket: {bucket_name}/{object_key}")

Image saved to S3 bucket: video2-analysis-bucket/processed-images/processed_image.jpg


## AWS Service Initialization and CloudWatch Logging

This code snippet demonstrates the initialization of AWS Rekognition and CloudWatch Logs clients, and sets up a logging mechanism for an image processing pipeline:

1. **Rekognition Client**: Initializes the AWS Rekognition client for image analysis.

2. **CloudWatch Logs Client**: Sets up a client for logging to CloudWatch.

3. **Log Group and Stream**: Creates a log group and stream if they don't exist.

4. **Logging Function**: Defines a function to send log messages to CloudWatch.

5. **Process Logging**: Logs each step of the image processing workflow, including:
   - Image loading
   - Rekognition label detection
   - Font loading for drawing labels
   - Drawing bounding boxes and labels
   - Saving the processed image to S3

This setup ensures comprehensive logging of the entire image processing pipeline, facilitating monitoring and debugging in a production environment.

In [7]:
# Initialize the Rekognition client
client = boto3.client('rekognition', region_name='us-east-1',
                      aws_access_key_id=access_key_id,
                      aws_secret_access_key=secret_access_key)

# Initialize the CloudWatch Logs client
logs_client = boto3.client('logs', region_name='us-east-1',
                           aws_access_key_id=access_key_id,
                           aws_secret_access_key=secret_access_key)

log_group = 'ImageProcessingLogs'  
log_stream = 'ImageProcessingStream'  


def create_log_group_if_not_exists():
    try:
        logs_client.describe_log_groups(logGroupNamePrefix=log_group)
    except logs_client.exceptions.ResourceNotFoundException:
        logs_client.create_log_group(logGroupName=log_group)
        print(f"Log group {log_group} created.")

def create_log_stream_if_not_exists():
    try:
        logs_client.describe_log_streams(logGroupName=log_group, logStreamNamePrefix=log_stream)
    except logs_client.exceptions.ResourceNotFoundException:
        logs_client.create_log_stream(logGroupName=log_group, logStreamName=log_stream)
        print(f"Log stream {log_stream} created.")

# Ensure the log group and stream exist
create_log_group_if_not_exists()
create_log_stream_if_not_exists()


def log_to_cloudwatch(message):
    timestamp = int(round(time.time() * 1000))  # Current timestamp in milliseconds
    logs_client.put_log_events(
        logGroupName=log_group,
        logStreamName=log_stream,
        logEvents=[{
            'timestamp': timestamp,
            'message': message
        }]
    )

# Step 1: Load the image and log the process
log_to_cloudwatch("Loading image file...")

with open(photo, 'rb') as image_file:
    source_bytes = image_file.read()

log_to_cloudwatch(f"Image file {photo} loaded successfully.")

# Step 2: Call AWS Rekognition to detect labels and log the result
log_to_cloudwatch("Calling Rekognition for label detection...")

try:
    detect_objects = client.detect_labels(Image={'Bytes': source_bytes})
    log_to_cloudwatch("Rekognition label detection completed.")
except Exception as e:
    log_to_cloudwatch(f"Error in Rekognition label detection: {str(e)}")
    raise

# Step 3: Load the font for drawing and log
log_to_cloudwatch("Loading font for label drawing...")

font_path = Path('/Users/pawanbtw/Downloads/Roboto_Condensed/RobotoCondensed-Italic-VariableFont_wght.ttf')
font = ImageFont.truetype(str(font_path), 30)

log_to_cloudwatch("Font file loaded successfully.")

# Step 4: Open the image and draw bounding boxes, then log the process
log_to_cloudwatch("Drawing bounding boxes and labels on image...")

image = Image.open(io.BytesIO(source_bytes))
draw = ImageDraw.Draw(image)

for label in detect_objects['Labels']:
    for instance in label.get('Instances', []):
        if 'BoundingBox' in instance:
            box = instance["BoundingBox"]
            
            left = image.width * box['Left']
            top = image.height * box['Top']
            width = image.width * box['Width']
            height = image.height * box['Height']
            
            points = (
                (left, top),
                (left + width, top),
                (left + width, top + height),
                (left, top + height),
                (left, top)
            )
            draw.line(points, width=5, fill="#16ded7")
            shape = [(left - 2, top - 35), (left + width + 2, top)]
            draw.rectangle(shape, fill="#16ded7")
            draw.text((left + 10, top - 30), label["Name"], fill="#000000", font=font)

log_to_cloudwatch("Bounding boxes and labels drawn on the image.")

# Step 5: Save the processed image to S3 and log the process
log_to_cloudwatch("Saving processed image to S3...")

s3_client = boto3.client('s3', region_name='us-east-1', 
                         aws_access_key_id=access_key_id,
                         aws_secret_access_key=secret_access_key)

# Convert the processed image back to bytes
image_byte_array = io.BytesIO()
image.save(image_byte_array, format="JPEG")
image_byte_array.seek(0)  # Rewind the byte stream to the beginning

# Define the S3 bucket and the folder path within the bucket
bucket_name = 'video2-analysis-bucket'  # Replace with your bucket name
folder_name = 'processed-images'     # The folder where the image will be saved
file_name = 'processed_image.jpg'    # The name you want to assign to the file in S3

# Construct the S3 object key (path within the bucket)
object_key = f"{folder_name}/{file_name}"

# Upload the image to the S3 bucket
s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=image_byte_array, ContentType='image/jpeg')

log_to_cloudwatch(f"Image saved to S3 bucket: {bucket_name}/{object_key}")


In [9]:
with open('/Users/pawanbtw/Downloads/img-rekognition_accessKeys.csv', 'r') as file:
    next(file)
    reader = csv.reader(file)
    for line in reader:
        access_key_id = line[0]
        secret_access_key = line[1]

client = boto3.client('rekognition', region_name='us-east-1',
                      aws_access_key_id=access_key_id,
                      aws_secret_access_key=secret_access_key)


logs_client = boto3.client('logs', region_name='us-east-1',
                           aws_access_key_id=access_key_id,
                           aws_secret_access_key=secret_access_key)

log_group = 'ImageProcessingLogs'  
log_stream = 'ImageProcessingStream'  


def create_log_group_if_not_exists():
    try:
        logs_client.describe_log_groups(logGroupNamePrefix=log_group)
    except logs_client.exceptions.ResourceNotFoundException:
        logs_client.create_log_group(logGroupName=log_group)
        print(f"Log group {log_group} created.")

def create_log_stream_if_not_exists():
    try:
        logs_client.describe_log_streams(logGroupName=log_group, logStreamNamePrefix=log_stream)
    except logs_client.exceptions.ResourceNotFoundException:
        logs_client.create_log_stream(logGroupName=log_group, logStreamName=log_stream)
        print(f"Log stream {log_stream} created.")

create_log_group_if_not_exists()
create_log_stream_if_not_exists()

def log_to_cloudwatch(message):
    timestamp = int(round(time.time() * 1000))  # Current timestamp in milliseconds
    logs_client.put_log_events(
        logGroupName=log_group,
        logStreamName=log_stream,
        logEvents=[{
            'timestamp': timestamp,
            'message': message
        }]
    )


log_to_cloudwatch("AWS Access Keys loaded successfully.")


photo = '/Users/pawanbtw/Desktop/stevejobs2.jpg'

with open(photo, 'rb') as image_file:
    source_bytes = image_file.read()

log_to_cloudwatch(f"Image file {photo} loaded successfully.")

try:
    detect_objects = client.detect_labels(Image={'Bytes': source_bytes})
    log_to_cloudwatch("Rekognition label detection completed.")
except Exception as e:
    log_to_cloudwatch(f"Error in Rekognition label detection: {str(e)}")
    raise

font_path = Path('/Users/pawanbtw/Downloads/Roboto_Condensed/RobotoCondensed-Italic-VariableFont_wght.ttf')
font = ImageFont.truetype(str(font_path), 30)

log_to_cloudwatch("Font file loaded successfully.")

image = Image.open(io.BytesIO(source_bytes))
draw = ImageDraw.Draw(image)

for label in detect_objects['Labels']:
    for instance in label.get('Instances', []):
        if 'BoundingBox' in instance:
            box = instance["BoundingBox"]
            
            left = image.width * box['Left']
            top = image.height * box['Top']
            width = image.width * box['Width']
            height = image.height * box['Height']
            
            points = (
                (left, top),
                (left + width, top),
                (left + width, top + height),
                (left, top + height),
                (left, top)
            )
            draw.line(points, width=5, fill="#16ded7")

            shape = [(left - 2, top - 35), (left + width + 2, top)]
            draw.rectangle(shape, fill="#16ded7")
            draw.text((left + 10, top - 30), label["Name"], fill="#000000", font=font)

log_to_cloudwatch("Bounding boxes and labels drawn on the image.")

processed_image_path = '/tmp/processed_image.jpg'
image.save(processed_image_path)

log_to_cloudwatch("Processed image saved to temporary file.")
s3_client = boto3.client('s3', region_name='us-east-1',
                         aws_access_key_id=access_key_id,
                         aws_secret_access_key=secret_access_key)

bucket_name = 'video2-analysis-bucket'  # Replace with your S3 bucket name
file_name = f"processed_images/{uuid.uuid4().hex}.jpg"

try:
    s3_client.upload_file(processed_image_path, bucket_name, file_name)
    log_to_cloudwatch(f"Processed image uploaded to S3 at {bucket_name}/{file_name}")
except Exception as e:
    log_to_cloudwatch(f"Error uploading image to S3: {str(e)}")
    raise
log_to_cloudwatch(f"Processed image uploaded to S3 at {bucket_name}/{file_name}")


NameError: name 'uuid' is not defined